# Benchmark Capture

This notebook utilizes Sony's [Remote Camera API](https://developer.sony.com/cameras/) to automate
image capture for Triangulation Matting.

In [ ]:
import os
import requests
import io
import re
import shutil
from os import path
from imutils import paths
import glob
import time
import json
import tkinter as tk
from tkinter import simpledialog
from tkinter.colorchooser import askcolor
from PIL import Image, ImageTk
from libsonyapi.camera import Camera
from libsonyapi.actions import Actions
from datetime import datetime

def from_rgb(r, g, b):
    return f'#{r:02x}{g:02x}{b:02x}'

image_names = []

postview = "/home/jannes/uni/jk-masterarbeit/green_benchmark/postview"
raw_jpg = "/home/jannes/uni/jk-masterarbeit/green_benchmark/raw_jpg"
log_dir = "/home/jannes/uni/jk-masterarbeit/green_benchmark/logs"


1. Set the camera in remote mode.
2. Connect to the cameras Wi-Fi.
3. Run the code cell below to connect to the API and do some configuration.


In [ ]:
try:
    camera = Camera()
    camera_info = camera.info()
    print(camera_info)
except Exception as e:
    camera = None
    print(e)

def configure_camera():
    print("Start configuring camera...")
    print("Setting Manual Focus")
    camera.do(Actions.setFocusMode, "MF")
    print("Setting shooting mode: still")
    camera.do(Actions.setShootMode, "still")
    print("Setting contShootingMode mode: Single")
    camera.do(Actions.setContShootingMode, {"contShootingMode": "Single"})
    # camera.do(Actions.setStillQuality, "RAW+JPEG") # not supported
    print("Setting PostviewImageSize: Original")
    camera.do(Actions.setPostviewImageSize, "Original")
    print("Finished configuring.")

def auto_fokus():
    camera.do(Actions.setFocusMode, "AF-S")
    camera.do(Actions.actHalfPressShutter)
    # camera.do(Actions.cancelHalfPressShutter)
    pass

def take_picture():
    if camera:
        result = camera.do(Actions.actTakePicture)
    else:
        result = None
    return result

def set_aperture():
    camera.do(Actions.setFNumber, "2.8")
    pass

def set_shutter_speed(shutter_speed:str):
    if camera:
        result = camera.do(Actions.setShutterSpeed, shutter_speed)
    else:
        result = None
    return result

def get_availiable_shutter_speed():
    result = camera.do(Actions.getAvailableShutterSpeed)
    return result

def ready_for_shoot():
    if camera:
        result = camera.do(Actions.awaitTakePicture)
        ready = "error" not in result
    else:
        ready = True
    return ready

if camera:
    configure_camera()
    result = take_picture()
    camera_info = camera.info()
    available_shutter = get_availiable_shutter_speed()
    print("Available Shutter Speeds:")
    print(available_shutter)



# Capturing GUI

Run the code cell below to start the Capturing  GUI.


In [ ]:
normal_shutter_speed = "1/20"
contour_shutter_speed = "1/1000"

colors = [
    # ("red256",       from_rgb(255,0,0)),
    # ("green256",     from_rgb(0,255,0)),
    # ("blue256",      from_rgb(0,0,255)),
    # ("white",        from_rgb(255,255,255)),
    ("red96",        from_rgb(96,0,0)),
    ("green96",      from_rgb(0,96,0)),
    ("blue96",       from_rgb(0,0,96)),
    ("white96",      from_rgb(96,96,96)),
    # ("red128",       from_rgb(128,0,0)),
    # ("green128",     from_rgb(0,128,0)),
    # ("blue128",      from_rgb(0,0,128)),
    # ("white128",     from_rgb(128,128,128)),
    ("black",        "black"),
    # ("red32",        from_rgb(15,0,0)),
    # ("green32",      from_rgb(0,15,0)),
    # ("blue32",       from_rgb(0,0,15)),
    # ("redContour",   from_rgb(255,0,0)),
    # ("greenContour", from_rgb(0,255,0)),
    # ("blueContour",  from_rgb(0,0,255)),
]

dir_path = postview
dir_split_count = len(dir_path.split("/"))
image_names = []

root = tk.Tk()
root.title("Benchmark Capturing")
color_window = tk.Toplevel(root)
color_window.title("Background")
root.geometry("1000x600")

lst = tk.Listbox(root, width=45)
lst.pack(side="left", fill=tk.Y)

namelist = list(paths.list_images(dir_path))
namelist.sort(key=os.path.getmtime)
for fname in namelist:
    lst.insert(0, "/".join(fname.split("/")[dir_split_count:]))

def get_img_width():
    return root.winfo_width() - lst.winfo_width()

lab = tk.Label(root)
lab.pack(side="left", fill=tk.BOTH)

def display_image(image, width=None, height=None):
    w = get_img_width()
    image.thumbnail((w, root.winfo_height()), Image.ANTIALIAS)

    photo = ImageTk.PhotoImage(image)
    lab.config(image=photo)
    lab.image = photo

lab.update()
lab.width = lab.winfo_width()
lab.height = lab.winfo_height()

def show_img(event):
    n = lst.curselection()
    fname = lst.get(n)
    image = Image.open(path.join(dir_path, fname))
    lab.image_copy = image.copy()
    display_image(image)

def resize_image(event):
    lab.width = event.width
    lab.height = event.height
    if hasattr(lab, "image_copy"):
        image = lab.image_copy.copy()
        display_image(image, lab.width, lab.height)

lab.bind('<Configure>', resize_image)
lst.bind("<<ListboxSelect>>", show_img)

def change_color(color):
    color_window['background'] = color

def wait(milliseconds):
    var = tk.IntVar()
    root.after(milliseconds, var.set, 1)
    root.wait_variable(var)

def shoot_pictures():
    now = datetime.now().strftime("%Y%m%d-%H%M%S")
    name = simpledialog.askstring("Input", "Name of the object", parent=root)
    img_dir = f"{name}_{now}"
    os.mkdir(path.join(dir_path, img_dir))

    picture_names = []
    for c_name, color in colors:
        change_color(color)
        # set_shutter_speed(contour_shutter_speed if "Contour" in c_name else normal_shutter_speed)
        while not ready_for_shoot():
            continue
        wait(200)
        result = take_picture()
        if result:
            picture_url:str = result["result"][0][0]
            picture_nr = picture_url.split("/")[-1]
            picture_nr = picture_nr.split(".")[0]
            picture_name = f"{img_dir}/{picture_nr}_{c_name}.JPG"

            picture_names.append(picture_name)
            response = requests.get(picture_url)
            raw_data = response.content
            img = Image.open(io.BytesIO(raw_data))
            img.save(path.join(dir_path, picture_name))
            if lst: lst.insert(0, picture_name)
            image_names.append(picture_name)
    # set_shutter_speed(normal_shutter_speed)

def choose_color():
    rgb, hex = askcolor(color=color_window["background"])
    return hex if hex is not None else "grey"

root.bind("<F11>", lambda event: color_window.attributes("-fullscreen",
                                    not color_window.attributes("-fullscreen")))
root.bind("<Escape>", lambda event: color_window.attributes("-fullscreen", False))

root.bind("#", lambda event: change_color(choose_color()))
root.bind("<g>", lambda event: change_color("green"))
root.bind("<r>", lambda event: change_color("red"))
root.bind("<b>", lambda event: change_color("blue"))
root.bind("<w>", lambda event: change_color("white"))
root.bind("<d>", lambda event: change_color("dark green"))
root.bind("<m>", lambda event: change_color("midnight blue"))
root.bind(",", lambda event: change_color("gray10"))
root.bind(".", lambda event: change_color("gray20"))
root.bind("-", lambda event: change_color("gray30"))
root.bind("<F1>", lambda event: shoot_pictures())

root.bind("1", lambda event: change_color(from_rgb(96,0,0)))
root.bind("2", lambda event: change_color(from_rgb(0,96,0)))
root.bind("3", lambda event: change_color(from_rgb(0,0,96)))
root.bind("4", lambda event: change_color(from_rgb(96,96,96)))
root.bind("5", lambda event: change_color(from_rgb(0,0,0)))

def on_closing():
    now = datetime.now().strftime("%Y%m%d-%H%M%S")
    if len(image_names) > 0:
        with open(path.join(log_dir, f"log_{now}.json"), 'w') as f:
            json.dump(image_names, f)
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)
root.mainloop()


To get the original images from the camera, connect it via cable and copy the files:

In [ ]:
path_to_imgs_on_camera = "/media/jannes/3337-3730/DCIM/100MSDCF"
log_file = "log_20230525-031719.json"

if log_file:
    with open(path.join(log_dir, log_file)) as f:
        image_names = json.load(f)

# camera_images = paths.list_files(path_to_imgs_on_camera)

for i, img_name in enumerate(image_names):
    img_nr = re.search("DSC\d+", img_name).group()
    jpg_cam = f"{img_nr}.JPG"
    jpg_cam_path = path.join(path_to_imgs_on_camera, jpg_cam)
    if not path.isfile(jpg_cam_path):
        print(jpg_cam_path, "not found. Skipping..")
        continue

    arw_cam = f"{img_nr}.ARW"
    arw_cam_path = path.join(path_to_imgs_on_camera, arw_cam)
    if not path.isfile(arw_cam_path):
        print(arw_cam_path, "not found. Only JPG will be copied.")

    os.makedirs(path.split(path.join(raw_jpg, img_name))[0], exist_ok=True)
    shutil.copyfile(jpg_cam_path, path.join(raw_jpg, img_name))
    shutil.copyfile(arw_cam_path, path.join(raw_jpg, str.replace(img_name, ".JPG", ".ARW")))